In [1]:
import jax
from jax import numpy as jp
from matplotlib.lines import Line2D
from matplotlib.patches import Circle
import matplotlib.pyplot as plt

import brax

In [2]:
print(brax.__version__)
import IPython
from IPython.display import HTML, clear_output
print(IPython.__version__)
from brax.io import model
from brax.io import json
from brax.io import html

0.9.0
8.13.2


In [3]:
from brax.io import mjcf

# ball = mjcf.loads(
#     """<mujoco>
#          <option timestep="0.005"/>
#          <worldbody>
#            <body pos="0 0 3">
#              <joint type="free"/>
#              <geom size="0.5 1 1" type="sphere"/>
#            </body>
#            <geom size="40 40 40" type="plane"/>
#            <geom size="0.5 0.5 0.5" pos="3 0 0.5" type="box" mass="1.0"/>
#          </worldbody>
#        </mujoco>
#   """)

In [4]:
ball = mjcf.loads("""
<mujoco>
	<option timestep="0.005"/>
	<worldbody>
		<geom name="floor0" pos="3 3 0" size="3 3 1" type="plane" condim="3"></geom>
		<body name="floor0" pos="3 3 0"></body>
		<body pos="0.15 0.15 0.15">
			<joint axis="1 0 0" damping="0.1" name="slide0" pos="0 0 0" type="slide"></joint>
			<joint axis="0 1 0" damping="0.1" name="slide1" pos="0 0 0" type="slide"></joint>
			<joint axis="0 0 1" damping="0.1" name="hinge0" pos="0 0 0" type="hinge"></joint>
			<geom name="agent" mass="1" pos="0 0 0" rgba="1 0 0 1" size="0.15" type="sphere" euler="1.57 0 0"></geom>
		</body>
		<body name="annotation:outer_bound" pos="0.15 0.15 0.15">
			<geom conaffinity="0" contype="0" mass="0" pos="0 0 0" rgba="0.417 0.7203 0.0001 0.0" size="0.0001 0.0001 0.0001" type="box"></geom>
		</body>
		<body name="moveable-box0" pos="0.25 0.25 0.25">
			<joint type="free" name="moveable-box0_free" damping="0.01" pos="0 0 0"></joint>
			<geom size="0.25 0.25 0.25" type="box" condim="3" name="moveable-box0" rgba="1 0.5 0.8 1" mass="1" friction="0.2"></geom>
		</body>
		<body name="moveable-box1" pos="0.25 0.25 0.25">
			<joint type="free" name="moveable-box1_free" damping="0.01" pos="0 0 0"></joint>
			<geom size="0.25 0.25 0.25" type="box" condim="3" name="moveable-box1" rgba="1 0.5 0.8 1" mass="1" friction="0.2"></geom>
		</body>
	</worldbody>
	<actuator>
		<motor gear="100" joint="slide0"></motor>
		<motor gear="100" joint="slide1"></motor>
	</actuator>
</mujoco>""")

In [5]:
print(ball.init_q)

[0.   0.   0.   0.25 0.25 0.25 1.   0.   0.   0.   0.25 0.25 0.25 1.
 0.   0.   0.  ]


In [6]:
#@title { run: "auto"}
from brax.positional import pipeline

elasticity = 0.85 #@param { type:"slider", min: 0.5, max: 1.0, step:0.05 }
ball_velocity = 1 #@param { type:"slider", min:-5, max:5, step: 0.5 }

# change the material elasticity of the ball and the plane
geoms = [g.replace(elasticity=jp.array([elasticity])) for g in ball.geoms]
ball = ball.replace(geoms=geoms)

# provide an initial velocity to the ball
qd = jp.zeros(ball.qd_size())
state = jax.jit(pipeline.init)(ball, ball.init_q, qd)

html.save('ball.html', ball, [state])

IndexError: list index out of range